In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from datetime import datetime, date
import warnings
from ipywidgets import widgets, interactive_output
from IPython.display import display
from source import fit_model
from sklearn.metrics import r2_score

warnings.filterwarnings("ignore")

In [85]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.feature_selection import  SelectKBest, RFE, f_regression, mutual_info_regression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import  DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from collections import Counter

In [4]:
df_expuestos = pd.read_csv("data/BD_Expuestos.csv", delimiter="\t", parse_dates=["FECHA_INICIO", "FECHA_CANCELACION", "FECHA_FIN"])

df_sociodem = pd.read_csv("data/BD_Sociodemograficas.csv", encoding="latin-1", delimiter="\t", parse_dates=["FechaNacimiento"])

# df_sin = pd.read_csv("data/DB_Siniestros.csv", parse_dates=["Mes_Pago"], date_format="%Y%m")
df_sin = pd.read_csv("data/DB_Siniestros_mod.csv", parse_dates=["Mes_Pago"], date_format="%Y-%m-%d")

caps = pd.read_excel("data/listas_de_causas.xlsx")

## Expuestos

In [5]:
# Cambiar los valores con una fecha fin anterior a la fecha de cancelacion
df_expuestos.loc[(df_expuestos["FECHA_CANCELACION"] > df_expuestos["FECHA_FIN"]), "FECHA_FIN"] = df_expuestos.loc[(df_expuestos["FECHA_CANCELACION"] > df_expuestos["FECHA_FIN"]), "FECHA_CANCELACION"] 

# Eliminar columna de fecha de cancelacion
df_expuestos.drop("FECHA_CANCELACION", axis=1, inplace=True)

## Siniestros

In [13]:
df_sin["Diagnostico_Codigo"] = df_sin["Diagnostico_Codigo"].str[0].replace(["W", "X"], ["Y"]*2) + df_sin["Diagnostico_Codigo"].str[1:]

tipo_cap = []

for i in df_sin["Diagnostico_Codigo"]:
    
    if i.isnumeric() or i == "-1":
        tipo_cap.append("Sin Diagnostico")
    else:
        
        for j in range(caps.shape[0]):
            if i[0] == caps.loc[j, "Codigos"][0] or i[0] == caps.loc[j, "Codigos"][4]:
                if int(i[1:3]) in range(int(caps.loc[j, "Codigos"][1:3]), int(caps.loc[j, "Codigos"][-2:])+1):
                    tipo_cap.append(caps.loc[j, "Cap"])
                    break
                else:
                    if i[0] == "H" or i[0] == "D":
                        continue
                    else:
                        tipo_cap.append("Sin info")

df_sin["Cap_CIE"] = tipo_cap  


KeyboardInterrupt



In [ ]:
df_sin.select_dtypes("object").apply(pd.Series.nunique).to_frame().rename({0:"Cantidad de categorias"}, axis=1).T

In [ ]:
fig = plt.figure(figsize=(7*((1+np.sqrt(5))/2), 7))

ax = plt.subplot(1, 3, 1)

data = df_sin.set_index("Mes_Pago").resample("M")["Valor_Pagado"].sum().to_frame()

df_sin.set_index("Mes_Pago").resample("M")["Valor_Pagado"].sum().to_frame().plot(kind="line", ax=ax, color=sns.color_palette("pastel")[0],
                                                                                title="Mes del Año vs. Valor pagado")
plt.xlabel("Mes de Pago")

ax = plt.subplot(1, 3, 2)

data = df_sin.set_index("Mes_Pago").resample("M")["Valor_Pagado"].sum().to_frame()

df_sin["Eventos"].plot(kind="box", ax=ax, color=sns.color_palette("pastel")[1], title="Boxplot para la distribucion\nde la variable Eventos")

ax = plt.subplot(1, 3, 3)

data = df_sin.set_index("Mes_Pago").resample("M")["Valor_Pagado"].sum().to_frame()

df_sin.groupby("Eventos", as_index=False)["Valor_Pagado"].sum().iloc[7:-1].plot(kind="scatter", x="Eventos", y="Valor_Pagado", ax=ax, color=sns.color_palette("pastel")[2], 
                                                                               title="Eventos vs. Valor Pagado")

plt.ylabel("Valor Pagado")

plt.tight_layout()

plt.show()


In [ ]:
wid_1 = widgets.Dropdown(options=np.sort(df_sin["Cap_CIE"].unique()), value="Sin Diagnostico")

def box_graphs(cod):

    df_sin.groupby("Cap_CIE").get_group(cod)["Eventos"].plot(kind="box");
    
output = interactive_output(box_graphs, {"cod":wid_1})
display(wid_1, output)

In [ ]:
data = df_sin.groupby(["Cap_CIE", "Eventos"], as_index=False)["Valor_Pagado"].sum()

fig = px.scatter(data, x="Eventos", y="Valor_Pagado", facet_col="Cap_CIE", facet_col_wrap=5, width=900*((1+np.sqrt(5))/2), height=900, template="plotly_white")

fig.for_each_annotation(lambda x: x.update(text= f"<b>{x.text.split('=')[1]}</b>"))


In [ ]:
w_1 = widgets.Dropdown(options=np.sort(df_sin["Diagnostico_Codigo"].unique()))

def groupby(w_1):

    df = df_sin[["Reclamacion", "Diagnostico_Codigo", "Diagnostico_Desc"]].groupby("Diagnostico_Codigo").get_group(w_1)
    display(df)

output = interactive_output(groupby, {"w_1":w_1})

display(output, w_1)

## Sociodemo



In [6]:
#Cálculo de la edad a partir de la fecha de nacimiento
df_sociodem["Edad"] = ((datetime(2019, 12, 31) - df_sociodem["FechaNacimiento"])/pd.Timedelta(days=365.25)).astype(int)

# Eliminar las filas donde la columna 'mujer' es -1
df_sociodem = df_sociodem[df_sociodem['Mujer'] != -1]

#Convertir variable 'Mujer' por 'Genero'
df_sociodem['Genero'] = df_sociodem['Mujer'].replace({0: 'hombre', 1: 'mujer'})

# Eliminar la columna Mujer, ya no es necesaria
df_sociodem = df_sociodem.drop(columns=['Mujer'])

In [ ]:
# Agrupar el DataFrame df_sociodem por "Ciudad" y "Edad", y sumar los valores de las columnas de enfermedades
data = df_sociodem.groupby(["Ciudad", "Edad"], as_index=False)[["CANCER", "EPOC", "DIABETES", "DIABETES", "ENF_CARDIOVASCULAR"]].sum()

# Convertir el DataFrame en formato largo para facilitar la visualización con Plotly
data = data.melt(id_vars=["Ciudad", "Edad"], value_vars=["CANCER", "EPOC", "DIABETES", "DIABETES", "ENF_CARDIOVASCULAR"], value_name="Total", var_name="Tipo")

# Crear una gráfica de líneas utilizando Plotly Express
fig = px.line(data, x="Edad", y="Total", facet_col="Ciudad", color="Tipo", template="plotly_white", facet_col_wrap=3, 
              labels={"Tipo":"<b>Tipo de enfermedad</b>"}, range_x=0, title="<b>Cantidad de enfermedades por edad,<br>en las distintas ciudades", 
              width=650*((1+np.sqrt(5))/2), height=650, color_discrete_sequence=px.colors.qualitative.T10)

# Actualizar el layout del título para centrarlo
fig.update_layout(title={"x":0.5})

# Modificar las anotaciones para resaltar el nombre de la ciudad
fig.for_each_annotation(lambda x: x.update(text= f"<b>{x.text.split('=')[1]}</b>"))

# Mostrar la gráfica
fig.show()

In [ ]:
df_sociodem["Ciudad"] = df_sociodem["Ciudad"].replace("Sin Información", np.nan)

data = df_sociodem["Ciudad"].value_counts().to_frame().reset_index()

plt.pie(data["count"], labels=data["Ciudad"], autopct="%1.1f%%", colors=sns.color_palette("hls", 9))

plt.title("Porcentaje de datos por Ciudad")

plt.show()

## Cruce de datos

In [165]:
df = df_sin.set_index("Asegurado_Id").join(df_sociodem.set_index("Asegurado_Id"), how="inner").join(df_expuestos.set_index("Asegurado_Id"), how="inner").reset_index()

df.dropna(inplace=True)

df["permanencia_poliza"] = ((df["FECHA_FIN"] - df["FECHA_INICIO"]).dt.days)

df["Eventos"].replace(0, 1, inplace=True)

df.drop(df[df["Reclamacion"].str.startswith("Sin")].index, inplace=True)

df["Cap_CIE"].replace("^Sin.*", "Sin Informacion", regex=True, inplace=True)

df.drop(df[df["Ciudad"].str.startswith("Sin")].index, inplace=True)




#Cambio de Variable respuesta

# Crear una nueva columna para sacar el promedio de lo que cuesta cada evento 
df['Costo_por_evento'] = (df['Valor_Pagado'] / df['Eventos']).round(2)

# Eliminar columnas sin informacion

columns_todrop = ["FECHA_FIN", "FECHA_INICIO", "FechaNacimiento", "Mes_Pago", "Diagnostico_Desc", "Diagnostico_Codigo", 
                  "Eventos"]

df.drop(columns_todrop, axis=1, inplace=True)




In [ ]:
wid_1 = widgets.Dropdown(options=df["Asegurado_Id"].unique()[:1000])

def groupby(id):
    data = df.groupby("Asegurado_Id").get_group(id)
    display(data)

output = interactive_output(groupby, {"id":wid_1})
display(wid_1, output)

In [ ]:
wid_1 = widgets.Dropdown(options=df["Reclamacion"].unique())

def groupby(id):
    data = df.groupby("Reclamacion").get_group(id)
    display(data)

output = interactive_output(groupby, {"id":wid_1})
display(wid_1, output)

In [ ]:
info = df["Reclamacion"].value_counts().to_frame().reset_index()

info["cumsum"] = (info["count"].cumsum()/info["count"].sum())*100


In [ ]:
fig = go.Figure([go.Bar(x=info["Reclamacion"], y=info["count"], yaxis="y1", name="", 
                        hovertemplate='<b>Reclamacion:</b> %{x}<br><b>Conteo Filas: </b>%{y}', 
                        marker={"color":"rgba(165, 82, 218, 0.48)"}), 
                 go.Scatter(x=info["Reclamacion"], y=info["cumsum"], yaxis="y2", name="", 
                            hovertemplate='<b>Porcentaje:</b> %{y:.2f}%<br><b>Reclamacion:</b> %{x}', 
                            marker={"color":"black"})])

fig.update_layout(xaxis={"fixedrange":True, "title":"<b>Reclamacion</b>"}, yaxis={"title":"<b>Conteo</b>", "fixedrange":True, "showgrid":False}, 
                  yaxis2={"fixedrange":True, "rangemode":"tozero", "overlaying":"y", "position":1, "side":"right",
                          "showgrid":False, "title":"<b>Porcentaje</b>", "tickvals":np.arange(0, 101, 20), "ticktext":[f'{i}%' for i in range(0, 101, 20)]},
                          showlegend=False, template="plotly_white", title={"text":"<b>Pareto Reclamaciones recurrentes<b>", "x":0.5})

fig

In [ ]:
info = df.groupby("Reclamacion", as_index=False)["Valor_Pagado"].sum().sort_values("Valor_Pagado", ascending=False)

info["cumsum"] = (info["Valor_Pagado"].cumsum()/info["Valor_Pagado"].sum())*100


In [ ]:
fig = go.Figure([go.Bar(x=info["Reclamacion"], y=info["Valor_Pagado"], yaxis="y1", name="", 
                        hovertemplate='<b>Reclamacion:</b> %{x}<br><b>Sum Valor Pagado: </b>%{y}', 
                        marker={"color":"rgba(165, 82, 218, 0.48)"}), 
                 go.Scatter(x=info["Reclamacion"], y=info["cumsum"], yaxis="y2", name="", 
                            hovertemplate='<b>Porcentaje:</b> %{y:.2f}%<br><b>Reclamacion:</b> %{x}', 
                            marker={"color":"black"})])

fig.update_layout(xaxis={"fixedrange":True, "title":"<b>Reclamacion</b>"}, yaxis={"title":"<b>Valor Pagado</b>", "fixedrange":True, "showgrid":False}, 
                  yaxis2={"fixedrange":True, "rangemode":"tozero", "overlaying":"y", "position":1, "side":"right",
                          "showgrid":False, "title":"<b>Porcentaje</b>", "tickvals":np.arange(0, 101, 20), "ticktext":[f'{i}%' for i in range(0, 101, 20)]},
                          showlegend=False, template="plotly_white", title={"text":"<b>Pareto valor pagado por reclamacion<b>", "x":0.5})

fig

In [ ]:
info = df.groupby("Reclamacion", as_index=False)["Costo_por_evento"].sum().sort_values("Costo_por_evento", ascending=False)

info["cumsum"] = (info["Costo_por_evento"].cumsum()/info["Costo_por_evento"].sum())*100


In [ ]:
fig = go.Figure([go.Bar(x=info["Reclamacion"], y=info["Costo_por_evento"], yaxis="y1", name="", 
                        hovertemplate='<b>Reclamacion:</b> %{x}<br><b>Suma Costo por evento: </b>%{y}', 
                        marker={"color":"rgba(165, 82, 218, 0.48)"}), 
                 go.Scatter(x=info["Reclamacion"], y=info["cumsum"], yaxis="y2", name="", 
                            hovertemplate='<b>Porcentaje:</b> %{y:.2f}%<br><b>Reclamacion:</b> %{x}', 
                            marker={"color":"black"})])

fig.update_layout(xaxis={"fixedrange":True, "title":"<b>Reclamacion</b>"}, yaxis={"title":"<b>Valor Pagado</b>", "fixedrange":True, "showgrid":False}, 
                  yaxis2={"fixedrange":True, "rangemode":"tozero", "overlaying":"y", "position":1, "side":"right",
                          "showgrid":False, "title":"<b>Porcentaje</b>", "tickvals":np.arange(0, 101, 20), "ticktext":[f'{i}%' for i in range(0, 101, 20)]},
                          showlegend=False, template="plotly_white", title={"text":"<b>Pareto valor pagado por reclamacion<b>", "x":0.5})

fig

### Eliminar categorias de la columna reclamacion

In [166]:
_ = df.groupby("Reclamacion", as_index=False)["Costo_por_evento"].sum().sort_values("Costo_por_evento", ascending=False)
_["cumsum"] = (_["Costo_por_evento"].cumsum()/_["Costo_por_evento"].sum())*100

df["Reclamacion"].replace(_.loc[_["cumsum"] > 92, 'Reclamacion'].values, "Otros", inplace=True)

In [ ]:
data = df[["Edad", "Ciudad"]]

fig = px.box(data, x="Ciudad", y="Edad", color="Ciudad", template="plotly_white", color_discrete_sequence=px.colors.qualitative.Set1,
             labels={"Ciudad":'<b>Ciudad</b>', "Edad":'<b>Edad</b>'}, title="<b>Box plots de edad por ciudad</b>")

fig.update_layout(yaxis={"showgrid": False}, title={"x":0.5})

### Anadir columna de rango etario

In [167]:
df.drop(df[(df["Edad"] > 80) | (df["Edad"] < 0)].index, inplace=True)

rangos_etarios = {
    "Infancia": "0-5",
    "Niñez": "6-12",
    "Adolescencia": "13-18",
    "Juventud": "19-25",
    'Adultez Joven': "26-34",
    'Adultez Media': "35-49",
    'Adultez Tardía': "50-64",
    'Vejez': "65-79",
    "Ancianidad": "80-100"
}

letarios = []

for k in df["Edad"]:
    for i, j in rangos_etarios.items():
        l_sup = int(j.split("-")[-1])
        l_inf = int(j.split("-")[0])
        
        if (k >= l_inf) & (k <= l_sup):
            letarios.append(i)
            break
    

df["Rango_etario"] = letarios

In [ ]:
df.groupby("permanencia_poliza", as_index=False)["Costo_por_evento"].sum().plot(kind="scatter", x="permanencia_poliza", y="Costo_por_evento");

In [ ]:
data = df[["Costo_por_evento", "Rango_etario"]]

fig = px.box(data, x="Rango_etario", y="Costo_por_evento", color="Rango_etario", template="plotly_white", color_discrete_sequence=px.colors.qualitative.Set1,
             labels={"Rango_etario":'<b>Rango_etario</b>', "Edad":'<b>Edad</b>'}, title="<b>Box plots de edad por ciudad</b>")

fig.update_layout(yaxis={"showgrid": False}, title={"x":0.5})

## Creacion de sub data frames

In [10]:
names = [f'df_{i}' for i in  df["Ciudad"].unique()]

for i, j in zip(names, df["Ciudad"].unique()):
    locals().update({i:df.groupby("Ciudad").get_group(j).drop("Ciudad", axis=1)})

# Medellin

### Modelos todas las variables

In [93]:
# data = df_Medellin.drop("Rango_etario", axis=1).sample(100000)
data = df_Medellin.drop("Rango_etario", axis=1)
results = {}

In [99]:
df = data.copy()  

# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]


X = df.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df[target]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cat_processor = OneHotEncoder()


# Preprocesador de variables  numericas
num_processor = StandardScaler()

# DataFrame de variables numericas  
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()


processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

model = make_pipeline(processor, GradientBoostingRegressor())

model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['Reclamacion', 'Cap_CIE',
                                                   'Genero']),
                                                 ('num', StandardScaler(),
                                                  ['CANCER', 'EPOC', 'DIABETES',
                                                   'HIPERTENSION',
                                                   'ENF_CARDIOVASCULAR', 'Edad',
                                                   'permanencia_poliza'])])),
                ('gradientboostingregressor', GradientBoostingRegressor())])

In [100]:
pred_med = model.predict(X_test)
results[model.named_steps["gradientboostingregressor"].__class__.__name__] = [r2_score(y_test, pred_med), mean_squared_error(y_test, pred_med), mean_absolute_error(y_test, pred_med)]
results_med = pd.DataFrame(results, index=[0, 1, 2])
results_med

,SVR,RandomForestRegressor,GradientBoostingRegressor
0,-1.022575e-02,2.041262e-01,2.157780e-01
1,5.440423e+12,4.286062e+12,4.223313e+12
2,3.799040e+05,3.742062e+05,3.627525e+05


### Kbest

In [23]:
# df = df.sample(10000)  
df_vars = data.copy()  

plot = False
# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]

# X = df.drop(to_drop, axis=1)


X = df_vars.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df_vars[target]

# Division de datos de evaluacion y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Preprocesador de variables categoricas
cat_processor = OrdinalEncoder()
# Preprocesador de variables numericas
num_processor = MinMaxScaler()

# DataFrame de variables numericas
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()

# Union de procesadores
processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

# Dicionarion para guardar resultados del algoritmo Kbest (una columna por metrica)
vars = {"f_regression":[], "mutual_info":[]}

# Evaluacion de algoritmo Kbest con diferentes metricas y diferentes k
for m in (f_regression, mutual_info_regression):
  # for k in range(5, 10):
    k = 9

    # Pipeline con el selector y preprocesador 
    selector = make_pipeline(processor,
                          SelectKBest(m, k=k))

    # Entrenar selector
    selector.fit(X_train, y_train)

    # Asignar lista con variables escogidas con cada kbest al valor correspondiente en el diccionario
    if m == f_regression:
      # Asignar listas a la llave f_regression
      vars["f_regression"] += selector.get_feature_names_out().tolist()
    else:
      # Asignar listas a la llave mutual_info
      vars["mutual_info"] += selector.get_feature_names_out().tolist()

# Crear data frame con los index como las variables y los valores de veces escogidas por cada metrica
vars_kb = pd.DataFrame({i:pd.Series(j).value_counts() for i,j in vars.items()})
vars_kb.index = vars_kb.index.str[5:]

plot_metric = "mutual_info"
# Plot the DataFrame con variables y las veces escogidas
if plot:
    # Ordenar DataFrame segun metrica escogida
    vars_kb.sort_values(by=plot_metric, ascending=True).plot(kind="barh")
    # Ubicar legenda
    plt.legend(loc=[0.7, 0.2])
    plt.title("Feature Importance (f_regression vs mutual information)")
    plt.show()

In [24]:
pd.DataFrame(vars).map(lambda x: x[5:])

,f_regression,mutual_info
0,Reclamacion,Reclamacion
1,Cap_CIE,Cap_CIE
2,Genero,Genero
3,CANCER,CANCER
4,EPOC,EPOC
5,DIABETES,DIABETES
6,HIPERTENSION,HIPERTENSION
7,ENF_CARDIOVASCULAR,Edad
8,Edad,permanencia_poliza


### RFE

In [25]:
# df_vars = data.sample(100000)  
df_vars = data.copy()  



plot = False
# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]

# X = df.drop(to_drop, axis=1)


X = df_vars.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df_vars[target]

# Division de datos de evaluacion y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Preprocesador de variables categoricas
cat_processor = OrdinalEncoder()
# Preprocesador de variables numericas
num_processor = MinMaxScaler()

# DataFrame de variables numericas
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()

# Union de procesadores
processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

criterion = ["squared_error", "absolute_error"]

vars_rfe = {}
num_rfe = 9


c = "squared_error"
selector = make_pipeline(processor,
                         RFE(DecisionTreeRegressor(criterion=c, random_state=42), n_features_to_select=num_rfe))
    
# Entrenar selector
selector.fit(X_train, y_train)

# Guardar las variables seleccionadas con cada combianacion de criterios en las columnas    
vars_rfe[c] = X.columns[selector.named_steps["rfe"].support_].values

In [26]:
pd.DataFrame(vars_rfe)

,squared_error
0,Reclamacion
1,Cap_CIE
2,CANCER
3,EPOC
4,DIABETES
5,HIPERTENSION
6,ENF_CARDIOVASCULAR
7,Genero
8,permanencia_poliza


# Barranquilla

### Modelos con todas las variables

In [109]:
data = df_Barranquilla.drop("Rango_etario", axis=1)
results = {}

In [114]:
df = data.copy()  

# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]


X = df.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df[target]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cat_processor = OneHotEncoder()


# Preprocesador de variables  numericas
num_processor = StandardScaler()

# DataFrame de variables numericas  
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()


processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

model = make_pipeline(processor, GradientBoostingRegressor())

model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['Reclamacion', 'Cap_CIE',
                                                   'Genero']),
                                                 ('num', StandardScaler(),
                                                  ['CANCER', 'EPOC', 'DIABETES',
                                                   'HIPERTENSION',
                                                   'ENF_CARDIOVASCULAR', 'Edad',
                                                   'permanencia_poliza'])])),
                ('gradientboostingregressor', GradientBoostingRegressor())])

In [115]:
pred_barr = model.predict(X_test)
results[model.named_steps["gradientboostingregressor"].__class__.__name__] = [r2_score(y_test, pred_barr), mean_squared_error(y_test, pred_barr), mean_absolute_error(y_test, pred_barr)]
results_barr = pd.DataFrame(results, index=[0, 1, 2])
results_barr

,SVR,RandomForestRegressor,GradientBoostingRegressor
0,-1.017605e-02,1.573912e-01,2.447468e-01
1,4.854842e+12,4.049525e+12,3.629699e+12
2,3.528891e+05,3.532923e+05,3.414803e+05


### Kbest

In [28]:
# df = df.sample(10000)  
df_vars = data.copy()  

plot = False
# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]

# X = df.drop(to_drop, axis=1)


X = df_vars.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df_vars[target]

# Division de datos de evaluacion y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Preprocesador de variables categoricas
cat_processor = OrdinalEncoder()
# Preprocesador de variables numericas
num_processor = MinMaxScaler()

# DataFrame de variables numericas
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()

# Union de procesadores
processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

# Dicionarion para guardar resultados del algoritmo Kbest (una columna por metrica)
vars = {"f_regression":[], "mutual_info":[]}

# Evaluacion de algoritmo Kbest con diferentes metricas y diferentes k
for m in (f_regression, mutual_info_regression):
  # for k in range(5, 10):
    k = 9

    # Pipeline con el selector y preprocesador 
    selector = make_pipeline(processor,
                          SelectKBest(m, k=k))

    # Entrenar selector
    selector.fit(X_train, y_train)

    # Asignar lista con variables escogidas con cada kbest al valor correspondiente en el diccionario
    if m == f_regression:
      # Asignar listas a la llave f_regression
      vars["f_regression"] += selector.get_feature_names_out().tolist()
    else:
      # Asignar listas a la llave mutual_info
      vars["mutual_info"] += selector.get_feature_names_out().tolist()

# Crear data frame con los index como las variables y los valores de veces escogidas por cada metrica
vars_kb = pd.DataFrame({i:pd.Series(j).value_counts() for i,j in vars.items()})
vars_kb.index = vars_kb.index.str[5:]



In [29]:
pd.DataFrame(vars).map(lambda x: x[5:])

,f_regression,mutual_info
0,Reclamacion,Reclamacion
1,Cap_CIE,Cap_CIE
2,Genero,Genero
3,CANCER,EPOC
4,EPOC,DIABETES
5,DIABETES,HIPERTENSION
6,HIPERTENSION,ENF_CARDIOVASCULAR
7,ENF_CARDIOVASCULAR,Edad
8,Edad,permanencia_poliza


### RFE

In [30]:
# df_vars = data.sample(100000)  
df_vars = data.copy()  



plot = False
# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]

# X = df.drop(to_drop, axis=1)


X = df_vars.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df_vars[target]

# Division de datos de evaluacion y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Preprocesador de variables categoricas
cat_processor = OrdinalEncoder()
# Preprocesador de variables numericas
num_processor = MinMaxScaler()

# DataFrame de variables numericas
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()

# Union de procesadores
processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

criterion = ["squared_error", "absolute_error"]

vars_rfe = {}
num_rfe = 9



c = "squared_error"
selector = make_pipeline(processor,
                         RFE(DecisionTreeRegressor(criterion=c, random_state=42), n_features_to_select=num_rfe))
    
# Entrenar selector
selector.fit(X_train, y_train)

# Guardar las variables seleccionadas con cada combianacion de criterios en las columnas    
vars_rfe[c] = X.columns[selector.named_steps["rfe"].support_].values

In [31]:
pd.DataFrame(vars_rfe)

,squared_error
0,Reclamacion
1,Cap_CIE
2,CANCER
3,EPOC
4,DIABETES
5,HIPERTENSION
6,ENF_CARDIOVASCULAR
7,Genero
8,permanencia_poliza


# Bogota

### Modelos con todas las variables

In [123]:
data = df_Bogota.drop("Rango_etario", axis=1).sample(100000)
results = {}

In [127]:
df = data.copy()  

# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]


X = df.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df[target]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cat_processor = OneHotEncoder()


# Preprocesador de variables  numericas
num_processor = StandardScaler()

# DataFrame de variables numericas  
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()


processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

model = make_pipeline(processor, GradientBoostingRegressor())

model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['Reclamacion', 'Cap_CIE',
                                                   'Genero']),
                                                 ('num', StandardScaler(),
                                                  ['CANCER', 'EPOC', 'DIABETES',
                                                   'HIPERTENSION',
                                                   'ENF_CARDIOVASCULAR', 'Edad',
                                                   'permanencia_poliza'])])),
                ('gradientboostingregressor', GradientBoostingRegressor())])

In [128]:
pred_bog = model.predict(X_test)
results[model.named_steps["gradientboostingregressor"].__class__.__name__] = [r2_score(y_test, pred_bog), mean_squared_error(y_test, pred_bog), mean_absolute_error(y_test, pred_bog)]
results_bog = pd.DataFrame(results, index=[0, 1, 2])
results_bog

,RandomForestRegressor,GradientBoostingRegressor
0,8.873670e-02,2.246375e-01
1,6.762404e+12,5.753896e+12
2,3.901125e+05,3.755913e+05


### Kbest

In [33]:
# df = df.sample(10000)  
df_vars = data.copy()  

# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]

# X = df.drop(to_drop, axis=1)


X = df_vars.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df_vars[target]

# Division de datos de evaluacion y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Preprocesador de variables categoricas
cat_processor = OrdinalEncoder()
# Preprocesador de variables numericas
num_processor = MinMaxScaler()

# DataFrame de variables numericas
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()

# Union de procesadores
processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

# Dicionarion para guardar resultados del algoritmo Kbest (una columna por metrica)
vars = {"f_regression":[], "mutual_info":[]}

# Evaluacion de algoritmo Kbest con diferentes metricas y diferentes k
for m in (f_regression, mutual_info_regression):
  # for k in range(5, 10):
    k = 9

    # Pipeline con el selector y preprocesador 
    selector = make_pipeline(processor,
                          SelectKBest(m, k=k))

    # Entrenar selector
    selector.fit(X_train, y_train)

    # Asignar lista con variables escogidas con cada kbest al valor correspondiente en el diccionario
    if m == f_regression:
      # Asignar listas a la llave f_regression
      vars["f_regression"] += selector.get_feature_names_out().tolist()
    else:
      # Asignar listas a la llave mutual_info
      vars["mutual_info"] += selector.get_feature_names_out().tolist()

# Crear data frame con los index como las variables y los valores de veces escogidas por cada metrica
vars_kb = pd.DataFrame({i:pd.Series(j).value_counts() for i,j in vars.items()})
vars_kb.index = vars_kb.index.str[5:]

In [34]:
pd.DataFrame(vars).map(lambda x: x[5:])

,f_regression,mutual_info
0,Reclamacion,Reclamacion
1,Cap_CIE,Cap_CIE
2,Genero,Genero
3,CANCER,CANCER
4,EPOC,EPOC
5,DIABETES,DIABETES
6,HIPERTENSION,HIPERTENSION
7,ENF_CARDIOVASCULAR,Edad
8,Edad,permanencia_poliza


### RFE

In [35]:
# df_vars = data.sample(100000)  
df_vars = data.copy()  



plot = False
# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]

# X = df.drop(to_drop, axis=1)


X = df_vars.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df_vars[target]

# Division de datos de evaluacion y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Preprocesador de variables categoricas
cat_processor = OrdinalEncoder()
# Preprocesador de variables numericas
num_processor = MinMaxScaler()

# DataFrame de variables numericas
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()

# Union de procesadores
processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

criterion = ["squared_error", "absolute_error"]

vars_rfe = {}
num_rfe = 9

c = "squared_error"
selector = make_pipeline(processor,
                         RFE(DecisionTreeRegressor(criterion=c, random_state=42), n_features_to_select=num_rfe))
    
# Entrenar selector
selector.fit(X_train, y_train)

# Guardar las variables seleccionadas con cada combianacion de criterios en las columnas    
vars_rfe[c] = X.columns[selector.named_steps["rfe"].support_].values

In [36]:
pd.DataFrame(vars_rfe)

,squared_error
0,Reclamacion
1,Cap_CIE
2,CANCER
3,EPOC
4,DIABETES
5,HIPERTENSION
6,Edad
7,Genero
8,permanencia_poliza


# Cartagena

### Modelos con todas las variables

In [130]:
data.shape

(616225, 14)

In [140]:
data = df_Cartagena.drop("Rango_etario", axis=1)#.sample(100000)
# results = {}

In [141]:
df = data.copy()  

# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]


X = df.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df[target]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cat_processor = OneHotEncoder()


# Preprocesador de variables  numericas
num_processor = StandardScaler()

# DataFrame de variables numericas  
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()


processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

model = make_pipeline(processor, GradientBoostingRegressor())

model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['Reclamacion', 'Cap_CIE',
                                                   'Genero']),
                                                 ('num', StandardScaler(),
                                                  ['CANCER', 'EPOC', 'DIABETES',
                                                   'HIPERTENSION',
                                                   'ENF_CARDIOVASCULAR', 'Edad',
                                                   'permanencia_poliza'])])),
                ('gradientboostingregressor', GradientBoostingRegressor())])

In [142]:
pred_car = model.predict(X_test)
results[model.named_steps["gradientboostingregressor"].__class__.__name__] = [r2_score(y_test, pred_car), mean_squared_error(y_test, pred_car), mean_absolute_error(y_test, pred_car)]
results_car = pd.DataFrame(results, index=[0, 1, 2])
results_car

,RandomForestRegressor,GradientBoostingRegressor
0,-5.429865e-03,2.430433e-01
1,2.484551e+12,3.130394e+12
2,3.228915e+05,3.289939e+05


### Kbest

In [38]:
# df = df.sample(10000)  
df_vars = data.copy()  

# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]

# X = df.drop(to_drop, axis=1)


X = df_vars.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df_vars[target]

# Division de datos de evaluacion y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Preprocesador de variables categoricas
cat_processor = OrdinalEncoder()
# Preprocesador de variables numericas
num_processor = MinMaxScaler()

# DataFrame de variables numericas
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()

# Union de procesadores
processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

# Dicionarion para guardar resultados del algoritmo Kbest (una columna por metrica)
vars = {"f_regression":[], "mutual_info":[]}

# Evaluacion de algoritmo Kbest con diferentes metricas y diferentes k
for m in (f_regression, mutual_info_regression):
  # for k in range(5, 10):
    k = 9

    # Pipeline con el selector y preprocesador 
    selector = make_pipeline(processor,
                          SelectKBest(m, k=k))

    # Entrenar selector
    selector.fit(X_train, y_train)

    # Asignar lista con variables escogidas con cada kbest al valor correspondiente en el diccionario
    if m == f_regression:
      # Asignar listas a la llave f_regression
      vars["f_regression"] += selector.get_feature_names_out().tolist()
    else:
      # Asignar listas a la llave mutual_info
      vars["mutual_info"] += selector.get_feature_names_out().tolist()

# Crear data frame con los index como las variables y los valores de veces escogidas por cada metrica
vars_kb = pd.DataFrame({i:pd.Series(j).value_counts() for i,j in vars.items()})
vars_kb.index = vars_kb.index.str[5:]

In [39]:
pd.DataFrame(vars).map(lambda x: x[5:]) 

,f_regression,mutual_info
0,Reclamacion,Reclamacion
1,Cap_CIE,Cap_CIE
2,Genero,Genero
3,CANCER,EPOC
4,EPOC,DIABETES
5,DIABETES,HIPERTENSION
6,HIPERTENSION,ENF_CARDIOVASCULAR
7,ENF_CARDIOVASCULAR,Edad
8,Edad,permanencia_poliza


### RFE

In [40]:
# df_vars = data.sample(100000)  
df_vars = data.copy()  



plot = False
# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]

# X = df.drop(to_drop, axis=1)


X = df_vars.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df_vars[target]

# Division de datos de evaluacion y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Preprocesador de variables categoricas
cat_processor = OrdinalEncoder()
# Preprocesador de variables numericas
num_processor = MinMaxScaler()

# DataFrame de variables numericas
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()

# Union de procesadores
processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

criterion = ["squared_error", "absolute_error"]

vars_rfe = {}
num_rfe = 9


c = "squared_error"
selector = make_pipeline(processor,
                         RFE(DecisionTreeRegressor(criterion=c, random_state=42), n_features_to_select=num_rfe))
    
# Entrenar selector
selector.fit(X_train, y_train)

# Guardar las variables seleccionadas con cada combianacion de criterios en las columnas    
vars_rfe[c] = X.columns[selector.named_steps["rfe"].support_].values

In [41]:
pd.DataFrame(vars_rfe)

,squared_error
0,Reclamacion
1,Cap_CIE
2,CANCER
3,EPOC
4,DIABETES
5,HIPERTENSION
6,ENF_CARDIOVASCULAR
7,Genero
8,permanencia_poliza


# Cali

### Modelos con todas las variables

In [147]:
data = df_Cali.drop("Rango_etario", axis=1)#.sample(100000)
# results = {}

In [148]:
df = data.copy()  

# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]


X = df.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df[target]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cat_processor = OneHotEncoder()


# Preprocesador de variables  numericas
num_processor = StandardScaler()

# DataFrame de variables numericas  
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()


processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

model = make_pipeline(processor, GradientBoostingRegressor())

model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['Reclamacion', 'Cap_CIE',
                                                   'Genero']),
                                                 ('num', StandardScaler(),
                                                  ['CANCER', 'EPOC', 'DIABETES',
                                                   'HIPERTENSION',
                                                   'ENF_CARDIOVASCULAR', 'Edad',
                                                   'permanencia_poliza'])])),
                ('gradientboostingregressor', GradientBoostingRegressor())])

In [149]:
pred_cal = model.predict(X_test)
results[model.named_steps["gradientboostingregressor"].__class__.__name__] = [r2_score(y_test, pred_cal), mean_squared_error(y_test, pred_cal), mean_absolute_error(y_test, pred_cal)]
results_cal = pd.DataFrame(results, index=[0, 1, 2])
results_cal

,RandomForestRegressor,GradientBoostingRegressor
0,3.998372e-02,7.408486e-02
1,1.930164e+12,1.553744e+13
2,3.147757e+05,3.336079e+05


### Kbest

In [43]:
# df = df.sample(10000)  
df_vars = data.copy()  

# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]

# X = df.drop(to_drop, axis=1)


X = df_vars.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df_vars[target]

# Division de datos de evaluacion y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Preprocesador de variables categoricas
cat_processor = OrdinalEncoder()
# Preprocesador de variables numericas
num_processor = MinMaxScaler()

# DataFrame de variables numericas
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()

# Union de procesadores
processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

# Dicionarion para guardar resultados del algoritmo Kbest (una columna por metrica)
vars = {"f_regression":[], "mutual_info":[]}

# Evaluacion de algoritmo Kbest con diferentes metricas y diferentes k
for m in (f_regression, mutual_info_regression):
  # for k in range(5, 10):
    k = 9

    # Pipeline con el selector y preprocesador 
    selector = make_pipeline(processor,
                          SelectKBest(m, k=k))

    # Entrenar selector
    selector.fit(X_train, y_train)

    # Asignar lista con variables escogidas con cada kbest al valor correspondiente en el diccionario
    if m == f_regression:
      # Asignar listas a la llave f_regression
      vars["f_regression"] += selector.get_feature_names_out().tolist()
    else:
      # Asignar listas a la llave mutual_info
      vars["mutual_info"] += selector.get_feature_names_out().tolist()

# Crear data frame con los index como las variables y los valores de veces escogidas por cada metrica
vars_kb = pd.DataFrame({i:pd.Series(j).value_counts() for i,j in vars.items()})
vars_kb.index = vars_kb.index.str[5:]

In [44]:
pd.DataFrame(vars).map(lambda x: x[5:])

,f_regression,mutual_info
0,Reclamacion,Reclamacion
1,Cap_CIE,Cap_CIE
2,Genero,Genero
3,CANCER,CANCER
4,EPOC,EPOC
5,DIABETES,DIABETES
6,HIPERTENSION,HIPERTENSION
7,ENF_CARDIOVASCULAR,Edad
8,Edad,permanencia_poliza


### RFE

In [45]:
# df_vars = data.sample(100000)  
df_vars = data.copy()  



plot = False
# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]

# X = df.drop(to_drop, axis=1)


X = df_vars.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df_vars[target]

# Division de datos de evaluacion y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Preprocesador de variables categoricas
cat_processor = OrdinalEncoder()
# Preprocesador de variables numericas
num_processor = MinMaxScaler()

# DataFrame de variables numericas
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()

# Union de procesadores
processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

criterion = ["squared_error", "absolute_error"]

vars_rfe = {}
num_rfe = 9

c = "squared_error"
selector = make_pipeline(processor,
                         RFE(DecisionTreeRegressor(criterion=c, random_state=42), n_features_to_select=num_rfe))
    
# Entrenar selector
selector.fit(X_train, y_train)

# Guardar las variables seleccionadas con cada combianacion de criterios en las columnas    
vars_rfe[c] = X.columns[selector.named_steps["rfe"].support_].values

In [46]:
pd.DataFrame(vars_rfe)

,squared_error
0,Reclamacion
1,Cap_CIE
2,CANCER
3,EPOC
4,DIABETES
5,HIPERTENSION
6,ENF_CARDIOVASCULAR
7,Genero
8,permanencia_poliza


# Todos los datos

In [168]:
df.head()

,Asegurado_Id,Reclamacion,Valor_Pagado,Cap_CIE,Ciudad,CANCER,EPOC,DIABETES,HIPERTENSION,ENF_CARDIOVASCULAR,Edad,Genero,Poliza_Asegurado_Id,permanencia_poliza,Costo_por_evento,Rango_etario
0,738284,CONSULTA EXTERNA,4.425121e+05,Sin Informacion,Cali,0,0,0,0,0,52,hombre,89612446,31,442512.13,Adultez Tardía
1,738284,Otros,4.137496e+05,Sin Informacion,Cali,0,0,0,0,0,52,hombre,89612446,31,206874.78,Adultez Tardía
2,738284,Otros,3.885089e+05,Sin Informacion,Cali,0,0,0,0,0,52,hombre,89612446,31,388508.94,Adultez Tardía
3,738284,EXAMENES DE DIAGNOSTICO,4.949378e+05,Sin Informacion,Cali,0,0,0,0,0,52,hombre,89612446,31,494937.78,Adultez Tardía
4,738284,LABORATORIO CLINICO,1.280335e+06,Sin Informacion,Cali,0,0,0,0,0,52,hombre,89612446,31,75313.82,Adultez Tardía


In [169]:
# data = df_Medellin.drop("Rango_etario", axis=1).sample(100000)
data = df.drop("Rango_etario", axis=1)
results = {}

In [170]:
df = data.copy()  

# Definir la variable objetivo
target = "Costo_por_evento"

to_drop = ["Asegurado_Id", "Poliza_Asegurado_Id", "Valor_Pagado", target]


X = df.drop(to_drop, axis=1)

# Convertir la variable objetivo a horas
y = df[target]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cat_processor = OneHotEncoder()


# Preprocesador de variables  numericas
num_processor = StandardScaler()

# DataFrame de variables numericas  
num_vals = X._get_numeric_data().columns.tolist()
# DataFrame de variables categoricas
cat_vals = X.select_dtypes("object").columns.tolist()


processor = ColumnTransformer(transformers=[("cat", cat_processor, cat_vals), ("num", num_processor, num_vals)])

model = make_pipeline(processor, GradientBoostingRegressor())

model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['Reclamacion', 'Cap_CIE',
                                                   'Ciudad', 'Genero']),
                                                 ('num', StandardScaler(),
                                                  ['CANCER', 'EPOC', 'DIABETES',
                                                   'HIPERTENSION',
                                                   'ENF_CARDIOVASCULAR', 'Edad',
                                                   'permanencia_poliza'])])),
                ('gradientboostingregressor', GradientBoostingRegressor())])

In [171]:
pred_total = model.predict(X_test)
results[model.named_steps["gradientboostingregressor"].__class__.__name__] = [r2_score(y_test, pred_total), mean_squared_error(y_test, pred_total), mean_absolute_error(y_test, pred_total)]
results_total = pd.DataFrame(results, index=[0, 1, 2])
results_total

,GradientBoostingRegressor
0,1.206904e-01
1,9.512656e+12
2,3.391549e+05


In [207]:
pd.Series(pred_total)

X_test.reset_index()

excel_pred = pd.concat([X_test.reset_index(), pd.Series(pred_total, name="costo")], axis=1)

# os.makedirs("predicciones")
excel_pred.to_excel("predicciones/predicciones_totales.xlsx", index=False)

In [190]:
fitted_processor = model.named_steps['columntransformer']

# Initialize an empty list to store the feature names
feature_names = []

# Loop through each transformer to get the feature names
for name, transformer, columns in fitted_processor.transformers_:
    if name == 'remainder' and transformer == 'passthrough':
        feature_names.extend(columns)
    elif name == 'num':
        feature_names.extend(columns)
    elif name == 'cat' and isinstance(transformer, OneHotEncoder):
        ohe_feature_names = transformer.get_feature_names_out(columns)
        feature_names.extend(ohe_feature_names)

# Get feature importances from the GradientBoostingRegressor
importances = model.named_steps["gradientboostingregressor"].feature_importances_

# Create a DataFrame to map feature names to their importances
feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# Sort the DataFrame by importance
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)


In [191]:
feature_importances_df

,Feature,Importance
9,Reclamacion_TRATAMIENTO QUIRURGICO HOSPITALARI...,0.458929
8,Reclamacion_TRATAMIENTO MEDICO HOSPITALARIO PO...,0.083863
1,Reclamacion_CIRUGIA AMBULATORIA POR ENFERMEDAD,0.058845
44,Edad,0.058379
7,Reclamacion_TRATAMIENTO AMBULATORIO POR CANCER,0.050974
14,Cap_CIE_IX,0.047532
15,Cap_CIE_Sin Informacion,0.037582
4,Reclamacion_EXAMENES DE DIAGNOSTICO,0.034753
11,Cap_CIE_II,0.025610
0,Reclamacion_ATENCION DEL PARTO,0.018961
